In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm.notebook import tqdm
from scipy.stats import bernoulli

In [2]:
class Model(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features=2, out_features=hidden_dim)
        self.fc2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.fc3 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.fc4 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.fc5 = nn.Linear(in_features=hidden_dim, out_features=1)

    def forward(self, x,t):
        x = torch.concat([x,t], axis=1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = F.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

In [4]:
train_dataset = torch.load('uxt_data_with_noise_Train.pt')
test_dataset = torch.load('uxt_data_without_noise_Test.pt')

u_train, x_train, t_train = train_dataset.T
x_train = x_train.unsqueeze(1)
t_train = t_train.unsqueeze(1)

u_test, x_test, t_test = test_dataset.T
x_test = x_test.unsqueeze(1)
t_test = t_test.unsqueeze(1)


In [5]:
def compute_grad(u, x):
    return_value =  torch.autograd.grad(u, x, create_graph=True, grad_outputs=torch.ones_like(u), allow_unused=True)[0]
    if return_value == None:
        return_value = torch.zeros_like(u)
    return return_value


def get_Laplacian(u,x):
    du_dx = compute_grad(u, x)
    d2u_dx2 = compute_grad(du_dx[:,0], x)[:,0]
    return d2u_dx2

def get_Error(u_pred, u_true):
    den = torch.linalg.norm(u_pred-u_true)
    nom = torch.linalg.norm(u_true)
    return float(den/nom)

In [6]:
num_epochs = 10000
learning_rate = 0.001
sch_Step_Size = 1000
sch_Gamma = 0.9
#Batch Size -> Use Full Batch
#The number of pts
num_GE = 5000 #Govern_Eqn
num_BC = 1000 #Boundary_Condition
num_IC = 1000 #Initial_Condition

#terminal time and the boundary of x
T = 2*torch.pi
x_min = -1.
x_max = 1.

In [7]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=sch_Step_Size, gamma=sch_Gamma)

logs = {'Train_Loss':[], 'Train_Err':[], 'Test_Err':[]}
for e in tqdm(range(1, num_epochs+1)):
    model.train()
    ### 01. Data Generation (Collocation Points) ###
    #points for GE
    x_GE = (T/2)*torch.rand(size=(num_GE,)) #(num_GE,)
    t_GE = T*torch.rand(size=(num_GE,)) #(num_GE,)
    x_GE = x_GE.unsqueeze(1)
    t_GE = t_GE.unsqueeze(1)
    # xt_GE = get_concat(x_GE, t_GE) #(num_GE, 2)

    #points for IC
    x_IC = (T/2)*torch.rand(size=(num_IC,))
    t_IC = torch.zeros_like(x_IC).float()
    x_IC = x_IC.unsqueeze(1)
    t_IC = t_IC.unsqueeze(1)
    # xt_IC = get_concat(x_IC, t_IC) #(num_IC, 2)

    #points for BC
    x_BC = (T/2)*torch.tensor(bernoulli.rvs(size=num_BC, p=0.5)).float()
    t_BC = T*torch.rand(size=(num_BC,))
    x_BC = x_BC.unsqueeze(1)
    t_BC = t_BC.unsqueeze(1)
    # xt_BC = get_concat(x_BC, t_BC) #of shape (num_BC, 2)


    ### 02. Update Parameters ###
    optimizer.zero_grad()

    #Loss_GE
    x_GE.requires_grad = True
    t_GE.requires_grad = True
    u_pred_GE = model(x_GE,t_GE).squeeze() #Forward; of shape (num_GE, )
    utt = get_Laplacian(u_pred_GE, t_GE).squeeze()
    uxx = get_Laplacian(u_pred_GE, x_GE).squeeze()
    Loss_GE = ((utt - uxx)**2).mean()

    #Loss_IC
    u_pred_IC = model(x_IC,t_IC).squeeze() #Forward; of shape (num_IC, )
    u_IC = torch.sin(x_IC).squeeze()
    Loss_IC = ((u_pred_IC - u_IC)**2).mean()

    #Loss_BC
    u_pred_BC = model(x_BC,t_BC).squeeze() #Forward; of shape (num_BC, )
    Loss_BC = (u_pred_BC**2).mean() #According to BC, u_pred_BC should be zero

    #Loss_NN
    u_prediction = model(x_train,t_train).squeeze() #Forward; of shape (num_train, )
    Loss_NN = ((u_prediction - u_train)**2).mean()

    #Loss_Total -> Update
    Loss_Total = Loss_GE + Loss_IC + Loss_BC + Loss_NN
    Loss_Total.backward()
    optimizer.step() #Backpropagation -> Update Parameters

    #Train_Err
    Train_Err = get_Error(u_prediction, u_train)

    #Log
    Loss_eth = float(Loss_Total.item())
    logs['Train_Loss'].append(Loss_eth)
    logs['Train_Err'].append(Train_Err)

    scheduler.step()

    #Evaluate Test Error
    model.eval()
    with torch.no_grad():
        u_prediction_test = model(x_test,t_test).squeeze() #Forward; of shape (num_test, )
        Test_Err = get_Error(u_prediction_test, u_test)
        logs['Test_Err'].append(Test_Err)

    #print()
    if e % 10 == 0:
        print('{}th Epoch Train Loss {:.5f}'.format(e, Loss_eth))
        print('{}th Epoch Train Error {:.3f}%'.format(e, 100*Train_Err))
        print('{}th Epoch Test Error {:.3f}%'.format(e, 100*Test_Err))
        print()

  0%|          | 0/10000 [00:00<?, ?it/s]

10th Epoch Train Loss 0.52396
10th Epoch Train Error 65.623%
10th Epoch Test Error 84.285%

20th Epoch Train Loss 0.36314
20th Epoch Train Error 51.351%
20th Epoch Test Error 83.876%

30th Epoch Train Loss 0.33536
30th Epoch Train Error 43.268%
30th Epoch Test Error 95.176%

40th Epoch Train Loss 0.28532
40th Epoch Train Error 39.935%
40th Epoch Test Error 88.175%

50th Epoch Train Loss 0.20933
50th Epoch Train Error 34.742%
50th Epoch Test Error 87.432%

60th Epoch Train Loss 0.14321
60th Epoch Train Error 23.905%
60th Epoch Test Error 88.026%

70th Epoch Train Loss 0.10550
70th Epoch Train Error 16.338%
70th Epoch Test Error 84.158%

80th Epoch Train Loss 0.08204
80th Epoch Train Error 13.417%
80th Epoch Test Error 82.263%

90th Epoch Train Loss 0.07005
90th Epoch Train Error 11.434%
90th Epoch Test Error 82.632%

100th Epoch Train Loss 0.06622
100th Epoch Train Error 10.604%
100th Epoch Test Error 76.998%

110th Epoch Train Loss 0.06347
110th Epoch Train Error 9.953%
110th Epoch Tes

In [8]:
torch.save(model.state_dict(), '1D_wave_eqn.pth')
from google.colab import files

files.download('1D_wave_eqn.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>